In the previous section, we began with a collection of implanted motifs that resulted in the following profile matrix.



If the strings in Dna were truly random, then we would expect that all nucleotides in the selected k-mers would be equally likely, resulting in an expected Profile in which every entry is approximately 0.25:



Such a uniform profile is essentially useless for motif finding because no string is more probable than any other according to this profile and because it does not provide any clues on what an implanted motif looks like.

At the opposite end of the spectrum, if we were incredibly lucky, we would choose the implanted k-mers Motifs from the very beginning, resulting in the first of the two profile matrices above. In practice, we are likely to obtain a profile matrix somewhere in between these two extremes, such as the following:



This profile matrix has already started to point us toward the implanted motif "ACGT", i.e., "ACGT" is the most likely 4-mer that can be generated by this profile. Fortunately, RandomizedMotifSearch is designed so that subsequent steps have a good chance of leading us toward this implanted motif (although it is not certain).

If you still doubt the efficacy of randomized algorithms, consider the following argument. We have already noticed that if the strings in Dna were random, then RandomizedMotifSearch would start from a nearly uniform profile, and there would be nothing to work with. However, the key observation is that the strings in Dna are not random because they include the implanted motif! These multiple occurrences of the same motif may direct the profile matrix away from the uniform profile and toward the implanted motif. For example, consider again the original randomly selected k-mers Motifs (shown in red):


You will see that the 4-mer "AGGT" in the last string happened to capture the implanted motif simply by chance. In fact, the profile formed from the remaining 4-mers ("taac", "GTct", "ccgG", and "acta") is uniform. Note that only completely captured motifs (like "AGGT") rather than partially captured motifs (like "GTct" or "ccgG") contribute to the statistical bias in the profile matrix.

You will see that the 4-mer "AGGT" in the last string happened to capture the implanted motif simply by chance. In fact, the profile formed from the remaining 4-mers ("taac", "GTct", "ccgG", and "acta") is uniform.

Unfortunately, capturing a single implanted motif is often insufficient to steer RandomizedMotifSearch to an optimal solution. Therefore, since the number of starting positions of k-mers is huge, the strategy of randomly selecting motifs is often not as successful as in the simple example above. The chance that these randomly selected k-mers will be able to guide us to the optimal solution is relatively small.

Note that RandomizedMotifSearch may change all t strings in Motifs in a single iteration. This strategy may prove reckless, since some correct motifs (captured in Motifs) may potentially be discarded at the next iteration. GibbsSampler is a more cautious iterative algorithm that discards a single k-mer from the current set of motifs at each iteration and decides to either keep it or replace it with a new one. This algorithm thus moves with more caution in the space of all motifs, as illustrated below.




Like RandomizedMotifSearch, GibbsSampler starts with randomly chosen k-mers in each of t DNA strings, but it makes a random choice at each iteration. It uses a list of t randomly selected k-mers Motifs to come up with another (hopefully better scoring) set of k-mers. In contrast with RandomizedMotifSearch, which defines new motifs as 


`Motifs(Profile(Motifs), Dna)`
GibbsSampler randomly selects an integer i between 0 and t-1 and then randomly changes a single k-mer Motif[i]. That is, GibbsSampler makes two random choices at each iteration. It uses random.randint(0, t-1) for the first choice (since all t strings in Dna are equally likely), but it does not make sense to use random.randint to choose a k-mer from Motif[i] because some k-mers are more likely than others. Indeed, each k-mer Pattern in Motif[i] may have a different probability Pr(Pattern, Profile) that it was generated by Profile.

We will illustrate how GibbsSampler works on the same strings Dna that we considered before. Imagine that, at the initial step, the algorithm has chosen the following 4-mers (shown in red) and has randomly selected the third string for removal. To be more precise, GibbsSampler does not really remove the third string; it ignores it at this particular step and may analyze it again in subsequent steps. 

This results in the following motif, count, and profile matrices.

Note that the profile matrix deviates only slightly from the uniform profile, making us wonder whether we have any chance to be steered toward the implanted motif. We now use this profile matrix to compute the probabilities of all 4-mers in the deleted string "ccgGCGTtag":

Note that all but two of these probabilities are zero. This situation is similar to the one we encountered with GreedyMotifSearch, and as before, we need to augment zero probabilities with small pseudocounts to avoid disastrous results.

Application of Laplace’s Rule of Succession to the count matrix above yields the following updated count and profile matrices:

After adding pseudocounts, the 4-mer probabilities in the deleted string "ccgGCGTtag" are recomputed as follows:

The question is how to model a biased seven-sided die in which the probability of rolling the i-th side of the die corresponds to the probability of the i-th 4-mer above. We would first like to rescale these numbers so that they sum to 1, as shown below.


To rescale a collection of probabilities (the sides of the die) so that these probabilities sum to 1, we will write a function called Normalize(Probabilities). This function takes a dictionary Probabilities whose keys are k-mers and whose values are the probabilities of these k-mers (which do not necessarily sum to 1). The function should divide each value in Probabilities by the sum of all values in  Probabilities, then return the resulting dictionary.

In [5]:
from numba import jit

In [43]:
# Input: A dictionary Probabilities, where keys are k-mers and values are the probabilities of these k-mers (which do not necessarily sum up to 1)
# Output: A normalized dictionary where the probability of each k-mer was divided by the sum of all k-mers' probabilities
@jit
#@jit(nopython=True)
#print ("Does not work with arrays of probabilites")
def Normalize(Probabilities):
    _sum_probs = 0.0 #Sum of all probabilities
    new_prob = Probabilities.copy()#Make a copy.
    
    for key in Probabilities.keys():
        _sum_probs += Probabilities[key] #add up all probabilites
        
    for key in Probabilities.keys():
        new_prob[key] = Probabilities[key] / _sum_probs   #Normalize values   
        
    return new_prob

In [10]:
test_input1 = {'A': 0.1, 'C': 0.1, 'G': 0.1, 'T': 0.1}
test_output1 = {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}

assert(Normalize(test_input1)==test_output1)

We now need to simulate rolling a die so that "ccgG" has probability 4/80, "cgGC" has probability 8/80, and so on. We will do so by generating a random number p between 0 and 1. If p is between 0 and 4/80, then it corresponds to "ccgG". If p is between 4/80 and 4/80 + 8/80, then it corresponds to "cgGC", etc.

How can we generate a random number between 0 and 1? In addition to the randint function, Python’s random module also includes a function called uniform(a, b) that generates a random floating point number (i.e., a decimal) between a and b. We can therefore generate our desired random number p by calling random.uniform(0, 1).

Code Challenge (3 points): Generalize this idea by writing a function WeightedDie(Probabilities). This function takes a dictionary Probabilities whose keys are k-mers and whose values are the probabilities of these k-mers. The function should return a randomly chosen k-mer key with respect to the values in Probabilities. Then add this function to Motifs.py.

(Note: the 1 at the beginning of the sample dataset is simply for grading purposes, so please ignore it.)

In [20]:
# first, import the random package
import random
# Input:  A dictionary Probabilities whose keys are k-mers and whose values are the probabilities of these kmers
# Output: A randomly chosen k-mer with respect to the values in Probabilities
@jit
def WeightedDie(Probabilities):
    key_list = Probabilities.keys()
    bounded_probabilities = Probabilities.copy()
    
    #Build an ascending list of values
    previous_value = 0.0 #Start with 0.0 Prob
    for key in key_list:
        bounded_probabilities[key] = previous_value + Probabilities[key]
        previous_value = bounded_probabilities[key]
    
    #Now we roll the dice to see which bound we fall in.
    previous_value = 0.0 #Start with 0.0 Prob
    rand_value = random.uniform(0,1) 
    for key in key_list:
        #If our choice is between 0.0 and 0.25, then return 'A', for example
        if (rand_value >= previous_value) and (rand_value < bounded_probabilities[key]):
            return key
        previous_value = bounded_probabilities[key] #We try the next value
    
    
    

In [21]:

WeightedDie(Normalize(test_input1))

{'A': 0.25, 'C': 0.5, 'G': 0.75, 'T': 1.0}
0.03128877656919238 0.0


'A'

Now that we can simulate a weighted die roll over a collection of probabilities of strings, we need to make this function into a subroutine of a larger function that randomly chooses a k-mer from a string Text based on a profile matrix profile.

In other words, after setting the length of text and declaring a blank dictionary,
```
    n = len(Text)
    probabilities = {} 
```
we range over all possible k-mers in Text, computing the probability of each one and placing this probability into a dictionary.
```
    for i in range(0,n-k+1):
        probabilities[Text[i:i+k]] = Pr(Text[i:i+k], profile)
```
We then normalize these probabilities using the Normalize(probabilities) subroutine, and then return the result of rolling a weighted die over this dictionary to produce a k-mer.
```
    probabilities = Normalize(probabilities)
    return WeightedDie(probabilities)
```
Code Challenge (3 points): Assemble this code into a function ProfileGeneratedString(Text, profile, k) that takes a string Text, a profile matrix profile , and an integer k as input.  It should then return a randomly generated k-mer from Text whose probabilities are generated from profile, as described above.  Then add this function to Motifs.py.

In [44]:
@jit
def Pr(Motif, Profile):
    prob=1.0
    for char in range(len(Motif)):
        prob *= Profile[Motif[char]][char] # What is the probability that this character is in this position?    
    return prob

"""#@jit
def Normalize(Probabilities):
    key_list = list(Probabilities.keys())
    print (key_list[0])
    k = len(Probabilities[key_list[0]])# How many entries are in the prob list?
    new_prob = Probabilities.copy()#Make a copy.
    
    for i in range(k):
        _sum_probs = 0.0 #Sum of all probabilities
    
        for key in key_list:
            _sum_probs += Probabilities[key][i] #add up all probabilites in this row.

        for key in key_list:
            new_prob[key][i] = Probabilities[key][i] / _sum_probs   #Normalize values   

    return new_prob"""

# Input:  A string Text, a profile matrix Profile, and an integer k
# Output: ProfileGeneratedString(Text, profile, k)
@jit
def ProfileGeneratedString(Text, profile, k):
    
    n = len(Text)
    probabilities = {} 
    
    for i in range(0,n-k+1):
        probabilities[Text[i:i+k]] = Pr(Text[i:i+k], profile)
    probabilities = Normalize(probabilities)
    return WeightedDie(probabilities)    



In [45]:
text1 = "AAACCCAAACCC"
profile1 = {'A': [0.5, 0.1], 'C': [0.3, 0.2], 'G': [0.2, 0.4], 'T': [0.0, 0.3]}
k1 = 2
out1 = "AC"

ProfileGeneratedString(text1, profile1, k1)

{'AA': 0.20833333333333331, 'AC': 0.625, 'CC': 0.875, 'CA': 1.0}
0.7805120257676518 0.0
0.7805120257676518 0.20833333333333331
0.7805120257676518 0.625


'CC'

Let’s assume that after “rolling the seven-sided die” represented by the function Die(Probabilities), we arrive at the Profile-randomly generated 4-mer GCGT (the fourth 4-mer in the deleted string). The deleted string "ccgGCGTtag" is now added back to the collection of motifs, and "GCGT" substitutes the previously chosen "ccgG" in the third string in Dna, as shown below. We then roll a (fair) five-sided die and randomly select the first string from Dna for removal.



After constructing the motif and profile matrices, we obtain the following:



Note that the profile matrix looks more biased toward the implanted motif than the previous profile matrix did.

We update the count and profile matrices with pseudocounts:



Then, we compute the probabilities of all 4-mers in the deleted string "ttACCTtaac":



When we rescale these numbers so that they sum to 1 and roll a seven-sided die based on the results, we arrive at the Profile-randomly generated k-mer "ACCT", which we add to the collection Motifs.

After rolling the five-sided die once again, we randomly select the fourth string for removal.




We further add pseudocounts and construct the resulting count and profile matrices:




We now compute the probabilities of all 4-mers in the deleted string "cactaACGAg":




We need to roll a seven-sided die to produce a Profile-randomly generated 4-mer. Assuming the most probable scenario in which we select "ACGA", we update the selected 4-mers as follows:

You can see that the algorithm is beginning to converge. Rest assured that a subsequent iteration will produce all implanted motifs after we select the second string in Dna (when the incorrect 4-mer "GTct" will likely change into the implanted (4, 1)-motif "ACGT").

STOP and Think: Note that in contrast to RandomizedMotifSearch, which always moves from higher to lower scoring Motifs, GibbsSampler may move from lower to higher scoring Motifs. Does this make sense?

Although GibbsSampler performs well in many cases, it may converge to a suboptimal solution, particularly for difficult search problems with elusive motifs. A local optimum is a solution that is optimal within a small neighboring set of solutions, which is in contrast to a global optimum, or the optimal solution among all possible solutions. Since GibbsSampler explores just a small subset of solutions, it may “get stuck” in a local optimum. For this reason, similarly to RandomizedMotifSearch, it should be run many times with the hope that one of these runs will produce the best-scoring motifs. Yet convergence to a local optimum is just one of many issues we must consider in motif finding; see DETOUR: Complications in Motif Finding for some other challenges.



Code Challenge (3 points): Now that you have seen how GibbsSampler works, implement it in Python as a final challenge in this chapter. Your function should take a parameter N corresponding to the number of iterations that we plan to run the program. Don't forget to use pseudocounts!

Note: If you have trouble implementing GibbsSampler, please see below, where we provide a description of GibbsSampler using pseudocode, a method of describing algorithms that is not dependent on a particular programming language like Python. If you are interested in learning more about how biological problems can be solved computationally, please check out our Bioinformatics Specialization or its print companion, Bioinformatics Algorithms: An Active Learning Approach, which uses pseudocode to discuss dozens of bioinformatics algorithms.
```
    GibbsSampler(Dna, k, t, N)
        randomly select k-mers Motifs = (Motif1, …, Motift) in each string from Dna
        ﻿BestMotifs ← Motifs
        for j ← 1 to N
            i ← randomly generated integer between 1 and t
            Profile ← profile matrix formed from all strings in Motifs except for Motifi
            Motifi ← Profile-randomly generated k-mer in the i-th string
            if Score(Motifs) < Score(BestMotifs)
                BestMotifs ← Motifs
        return BestMotifs
    ```

In [ ]:
def GibbsSampler(Dna, k, t , N):

In [47]:
k, t, N = 8, 5, 100
Dna1 =[ "CGCCCCTCTCGGGGGTGTTCAGTAAACGGCCA",
"GGGCGAGGTATGTGTAAGTGCCAAGGTGCCAG",
"TAGTACCGAGACCGAAAGAAGTATACAGGCGT",
"TAGATCAAGTTTCAGGTGCACGTCGGTGAACC",
"AATCCACCAGCTCCACGTGCAATGTTGGCCTA"]
# Possible Output: AACGGCCA AAGTGCCA TAGTACCG  AAGTTTCA ACGTGCAA